# 다층 신경망

## 준비

In [9]:
from torchvision import datasets, transforms

BATCH_SIZE = 32
train_dataset = datasets.FashionMNIST(
    root='./data',  # 데이터 저장 경로
    train=True,  # 학습용 데이터셋
    download=True,  # 데이터가 없으면 다운로드
    transform=transforms.ToTensor()  # 텐서로 변환
)

In [10]:
test_dataset = datasets.FashionMNIST(
    root='./data',
    train=False, # 테스트용 데이터셋(이 부분만 달라짐)
    download=True,
    transform=transforms.ToTensor()
)

In [11]:
from torch.utils.data import Dataset

class BinaryMNIST(Dataset):
    def __init__(self, dataset, class_1, class_2):
        self.dataset = dataset
        self.indices = [i for i, (_, label) in enumerate(dataset) 
                        if label in [class_1, class_2]] # 원하는 클래스의 인덱스만
        self.class_map = {class_1: 0, class_2: 1} # class_1은 0으로, class_2는 1로
        
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        image, label = self.dataset[self.indices[idx]] # idx번째 데이터
        return image, torch.tensor(self.class_map[label]) # 레이블 바꾸기
        # label.item() -> label

In [12]:
binary_train_dataset = BinaryMNIST(train_dataset, 3, 4)  # 2번과 6번의 이항 분류
binary_test_dataset = BinaryMNIST(test_dataset, 3, 4)

In [13]:
import numpy as np
from PIL import Image
x, y = train_dataset[0]  # 0번 데이터

In [14]:
x = x.squeeze()  # 텐서 모양 (1, 28, 28) -> (28, 28)
x = x.numpy()  # 텐서 -> 배열
x = x * 255  # 값의 범위 [0, 1] -> [0, 255]
x = x.astype(np.uint8)  # 자료형 float -> uint8
Image.fromarray(x)  # 이미지 보기

In [15]:
y

9

In [16]:
import torch
import torch.nn as nn

class BinaryClassifier(nn.Module):
    def __init__(self, input_size=784):  # MNIST의 경우 28*28=784
        super().__init__()
        self.flatten = nn.Flatten()  # 입력 이미지를 1차원으로 평탄화
        self.linear = nn.Linear(input_size, 32)  # 한 개의 출력 뉴런
        self.sigmoid = nn.Sigmoid()  # 시그모이드 활성화 함수
        self.linear = nn.Linear(input_size, 32)  # 한 개의 출력 뉴런
        self.sigmoid = nn.Sigmoid()  # 시그모이드 활성화 함수
        self.linear = nn.Linear(input_size, 1)  # 한 개의 출력 뉴런
        self.sigmoid = nn.Sigmoid()  # 시그모이드 활성화 함수        
    def forward(self, x): # 모형에 x가 입력되면
        x = self.flatten(x) # 위에서 정의한 레이어들을 
        x = self.linear(x) # 순서대로
        x = self.sigmoid(x) # 통과하여
        return x # 결과를 반환
model = BinaryClassifier()


In [ ]:
criterion = nn.BCELoss() # 이항분류이므로 BCELoss
optimizer = torch.optim.RAdam(model.parameters())
from torch.utils.data import DataLoader

BATCH_SIZE = 32

train_loader = DataLoader(
    binary_train_dataset,  # 훈련 데이터
    batch_size=BATCH_SIZE,  # 32개씩
    shuffle=True)  # 섞어서(미니배치마다 조합이 다양하도록)

test_loader = DataLoader(
    binary_test_dataset,  # 테스트 데이터
    batch_size=BATCH_SIZE,  # 32개씩
    shuffle=False)  # 섞지 말고
def fit(model, train_loader, criterion, optimizer, epochs):
    model.train()  # 훈련 모드
    for epoch in range(epochs):
        for i, (x, y) in enumerate(train_loader):  # 미니배치마다
            optimizer.zero_grad()  # 기울기 초기화
            y_hat = model(x)  # 모형 예측
            loss = criterion(y_hat, y.float().view_as(y_hat))  # 오차 계산
            loss.backward()  # 기울기 계산
            optimizer.step()  # 가중치 수정
            print(f'Epoch: {epoch}, Batch: {i}, Loss: {loss.item()}')
fit(model, train_loader, criterion, optimizer, epochs=3) # 전체 데이터를 3회 학습


In [18]:
import numpy as np
y_true = []  # 정답
y_prob = []  # 예측한 확률
with torch.no_grad():  # 그래디언트 계산 비활성화
    for x, y in test_loader:
        probs = model(x)
        y_true.extend(y.numpy())  # 정답에 추가
        y_prob.extend(probs.squeeze().numpy())  # 확률에 추가
y_prob = np.array(y_prob)  # 배열로 변환


In [19]:
threshold = 0.5  # 문턱값
y_pred = np.where(y_prob > threshold, 1, 0) # 문턱값보다 크면 1 아니면 0

# 혼동행렬 구하기
from sklearn.metrics import *
confusion_matrix(y_true, y_pred)


array([[937,  63],
       [ 55, 945]], dtype=int64)

## 다층신경망

오늘 할 내용은 이제 다층신경망 부터 할 건데 우리가 이제 아까 모델을 만들 때 모델을 만들 때 리니어만 제가 보통 레이어로 친다

그렇게 얘기를 드렸는데요

플렉튼 같은 거는 그냥 입력을 변형하는 거니까 이거는 레이어로 보통 안 치고 리니어로 치고 시그모이드 이거는 그냥 리니어에 붙어 있는 걸로 봅니다

그래서 이거 두 개는 하나로 그냥 한 덩어리로 봐요

결국 이제 이거는 레이어가 하나짜리인 신경망인데 이런 걸 단층신경망이라고 하고요 이제 이런 단층신경망이 사실상 하는 거는 뭐냐 하면은 우리가 어떤 데이터 공간이 있을 때 지금 예를 들어서 어 셔츠가 있고 코트가 있고 Never는 뭔 conditions 더 advanced井 sitting one 이러다 Woman entire Die affect 그런데 또ky 그어 가지고는 얘네를 구별할 수가 없어요

최소한 선을 이렇게 두 개를 그어 줘야 됩니다

그래서 이제 이런 문제가 되면은 선형분리가 안 된다

이런 얘기를 하게 되고요 그래서 이런 식으로 오른쪽처럼 돼 있는 거를 이제 비선형이라고 하는데 비선형 문제는 어떻게 풀 수 있느냐 선을 많이 그으면 되죠 선을 많이 그어 가지고 분리를 하면 됩니다

선을 하나 그어서 안 되니까 선을 여러 개 그어야 되는데 이거는 이제 넘어가고 선을 여러 개 긋는 방식이 다층신경망입니다

그래서 여기 그림을 보시면 이거 하나 하나는 선 하나 하나가 되는데 이 선들을 다음번에 다시 포개면 복잡한 형상을 만들 수 있는 거죠

그래서 그렇게 만드는 거를 이제 다층신경망이라고 하고요 그래서 우리가 이제 데이터를 입력을 받는 부분 이거를 이제 입력층이라고 하고 데이터가 이제 마지막으로 예측을 내보내는 부분을 출력층이라고 하고 그 사이에 끼는 부분을 은닉층이라고 합니다

그래서 은닉층이 하는 역할은 뭐냐 하면 어떤 과제 수행에 필요한 특징을 학습한다

예를 들면 사람 얼굴이다

이러면 사람 얼굴은 좀 단순하게 생각하면 눈 코 입이 합쳐져서 얼굴이 되는 거잖아요

그러면 실제로 그렇게 되는 건 아닌데 예를 들어서 얘는 뭔가 눈에 대해서 학습하고 얘는 코에 대해서 학습하고 얘는 입에 대해서 학습하면 얘가 얼굴에 대한 어떤 처리를 할 수 있고 이렇게 해주는 거죠

이게 어떤 실제로 이렇게 되는 건 아니지만 비유하자면 이런 부분 부분적인 특징을 은닉층에서 학습을 합니다

그래서 최종적으로 출력층은 은닉층에서 학습한 걸 바탕으로 뭔가 예측을 하는 거죠

그다음에 만약에 레이어가 이렇게 여러 개가 들어간다

그럼 입력이 들어가면 얘는 좀 입력에 가까운 쪽에서는 단순한 특징을 학습을 해요

예를 들면은 얘는 눈썹이라든지 눈동자라든지 입술이라든지 이런 걸 학습하고 얘는 그걸 바탕으로 뭔가 학습을 하니까 눈이라든지 코라든지 입이라든지 이런 걸 학습하고 얘는 그걸 바탕으로 하니까 표정이라든지 이런 인상이라든지 이런 걸 학습하고 얘는 그거보다 더 복잡한 거 그러니까 앞쪽에서 뭔가 더 단순한 처리를 해주면 뒷단에서 더 복잡한 처리를 할 수 있는 거죠

그래서 그런 식으로 학습을 하게 됩니다

그러면 우리가 이 은닉층을 어떻게 집어넣어야 되냐 하는 거는 이 이미지에 얼룩이 들어간다는 거죠

이 이미지에 얼룩이 들어간다는 거죠

이게 얼마나 다양한 특징이 있느냐

그다음에 그 특징들을 기반으로 얼마나 복잡하게 쌓아 올려야 되느냐에 따라서 은닉층의 형태가 결정이 됩니다

만약에 특징이 여러 가지다

그러면 은닉층에 이런 것이 더 많이 들어가야 될 거고 그다음에 그 위로 뭔가 쌓아 올려야 되는 게 많다

예를 들면 얼굴만 중요한 게 아니라 우리 팔다리도 해야 되고 배경도 해야 되고 그러면 팔다리는 또 따로 쌓아야 되고 얼굴도 따로 쌓아야 되고 얼굴하고 팔다리를 또 합쳐야 되고 하니까 이 뒤로도 들어가는 게 많아지겠죠

근데 이제 문제는 뭐냐면 항상 머신더닝에서 어떤 모델이 적당한 모델이냐 라는 거는 우리가 사전에 알 수가 없습니다

왜냐면 우리가 데이터를 봐도 우리가 모르거든요

그러면 적당한 모델은 어떻게 아냐면 해봐 가지고 잘 돌아가는 모델이 적당한 모델입니다

그래서 약간 거꾸로인데 데이터에 맞춰서 모델을 만들어야 되는데 우리가 데이터에 대한 파악이 안 되기 때문에 사람의 머리로는 파악이 안 돼요

그래서 어떻게 하냐면 모델을 다양하게 만들어 봐 가지고 제일 잘 돌아가는 모델이 데이터에 맞는 거겠죠

예를 들면 여러분 국을 끓이는데 소금을 얼마나 넣어도 되냐

소금 넣어보고 간이 맞으면 소금을 잘 넣은 겁니다 국을 끓였는데 싱겁다 그러면 소금을 더 넣으시면 되죠

모델을 만들었는데 성능이 안 나온다 모델을 더 키우시면 됩니다

그래서 다양한 시도를 통해서 성능이 제일 좋은 것으로 결정합니다

우리가 사전에 어떤 이론적으로 알 수가 없기 때문에 해보고 좋으면 그게 맞는 거다

그래서 이게 제가 머신러닝에 익숙치 않은 분들한테 이 얘기를 하면 굉장히 이상하게 생각을 하시더라고요

왜냐면 우리가 보통 학교에서 뭘 배울 때는 대부분 분야가 뭔가 이론적으로 이건 이렇고 이건 저러니까 이렇게 해야 된다

이런 식으로 접근을 하거든요

머신러닝에서는 물론 그런 경우도 있는데 대부분은 그냥 한번 해보세요 해보고 잘 되면 그게 맞는 겁니다

약간 이런 식이라서 좀 혼란스러워 하시는 경우가 되게 많은데 사실이 그렇습니다

그래서 머신러닝 논문이 생각보다 읽기가 되게 편해요

특히 딥러닝 논문이 읽기가 쉬운데 딥러닝 논문은 논문 전체 수식이 잘해야 한 줄 나오나 거의 수식도 없고 읽을 게 없습니다

딥러닝 논문은 보시면 엄청 읽기가 쉬워요

왜 읽기가 쉽냐 하면 결국에는 내용이 뭐냐면 이렇게 해보니까 잘 되더라고요

이게 끝입니다

그러니까 읽을 게 없어요

그래서 요즘에 보면 대학생들도 딥러닝 학회에다가 논문 내고 그러거든요 학부생들도 왜냐면 이론적으로 접근할 게 별로 없고 제가 돌려봤는데 잘 되던데요

약간 이런 수준으로 논문을 쓰니까 유명한 학회에도 그 정도 수준의 논문이 다 그 정도 수준입니다 이론적으로 파고드는 논문이 없어요

그러니까 우리도 특별히 그렇게 고민하실 게 없다

그냥 해보고 잘 되면 잘 되는구만

이렇게 생각하시면 됩니다

그래서 예전에 안드레 카파시라고

굉장히 유명한 사람인데 테슬라의 자율주행 부문 최고 책임자였던 사람이 있거든요

지금은 자기 사업하겠다고 퇴사했는데 그 사람 퇴사할 때 일론 머스크가 자기 트위터에다가 너무 아쉽다 이러면서 그동안 고마웠고 일론 머스크가 그런 사람이 아니잖아요

보통 남한테 고맙다고 하고 이럴 사람이 아닌데 그 사람 퇴사할 때는 너무 서운하고 고맙고 그 정도로 굉장히 유명한 이 분야의 유명한 사람입니다

근데 그 사람이 예전에 어떤 얘기했는지요?

제가 네 주소가 아쉽네요

이 분야의 유명한 사람은 아예 자기가 뭐 돌려놓고 모델 만드는 코드를 돌려놓고 원래 퇴근할 때 결과를 봤어야 되는데 깜빡 잊고 휴가를 갔다는 거예요

2주 동안 휴가를 갔다 와가지고 갔다 온 동안 컴퓨터는 계속 새로운 모델을 만들어서 돌리고 학습시키고 막 이렇게 지 혼자서 돌아가고 있었던 거죠

2주 동안 휴가 갔다 왔더니 자기 생각보다 성능이 되게 잘 나오더라

그래서 여러분도 오래 돌려보세요 이런 거를 조언이라고 하더라고요

그래서 이 바닥이 정말..

어떤 바닥인지 보여주는 그런..

그런 말이다

이런 생각 그러니까 여러분 수학 공부를 열심히 하십시오

이런 게 아니라 컴퓨터로 오래 돌려놓고 휴가를 가십시오 시행착오가 되게 중요한 분야입니다 머신러닝은 그래서 시행착오로 찾는다

이렇게 생각을 하시면 되고요 그래서 우리가 여기 코드가 있는데 사실은 여기도 그냥 바로 적용을 할 수 있습니다

어떻게 적용하면 되냐면 여기 레이어 하나라고 했잖아요

그리고 복사해서 밑에다 한번 더 붙여주시면 짜잔 레이어가 두 개가 됐죠

이렇게 하면 다층신경망입니다

별거 아니죠

층을 하나 더 쌓고 싶으면 어떻게 하면 되느냐

마지막 두 줄을 복사를 해서 한번 더 붙이면 됩니다

그럼 이제 3층이에요

그다음에 여기가 1, 1, 1인데 맨 마지막은 항상 1이어야 되거든요

왜냐면 우리가 결과적으로는 확률 하나가 나와야 되니까 근데 여기 중간에 있는 거는 16이어도 되고 32이어도 되고 큰 상관은 없습니다

이거는 이제 마음대로 하시면 되는데 보통은 요즘에는 중간은 숫자를 그래도 통일을 해줍니다

그래서 32, 32, 32 이렇게 가다가 맨 마지막만 1 이렇게 끝나면 됩니다

그래서 이게 다층신경망이에요

이런 식으로 만들면 됩니다

그럼 이제 몇 개 더 넣어야 되나요?

해보시면 되는 거죠

두 개도 넣어보고 세 개도 넣어보고 그래서 이제 우리 교환에 보면은 이제 고급 신경망이 있는 거에요

이제 고 다층신경망 예시코드가 있는데 조금 결과적으로는 그 얘기에요

그래서 리니어를 두 번 넣었죠

두 번 넣었는데 지금 보면은 앞에서 했던 플래튼이 어디 갔는지 없어졌고 그다음에 시그모이드도 어디 갔는지 없어졌습니다

없어진 건 아니고 우리가 이제 플래튼이나 시그모이드 같은 애들은 이닛에다가 정의를 해 줘도 되고 포워드에다가 정의를 해 줘도 되는데 그래서 이제 두 가지를 다 정의해 줘도 되는데 두 가지를 다 보여 드리기 위해서 이번에는 포워드로 옮겨 놨어요

그래서 이제 제일 처음에 보면은 포워드에 X.뷰 한 다음에 마이너스 1, 784 이렇게 돼 있는데 요 얘기가 뭐냐면 이게 플래튼이랑 똑같은 겁니다

우리가 이미지가 N장의 이미지가 가로 28 세로 28 이렇게 돼 있기 때문에 요거를 784로 바꿔주는 역할을 하고요 그 다음에 요 X가 리니어 1으로 들어가서 리니어 1으로 들어가서 리니어 1으로 들어가서 그 다음에 이제 원래는 아까는 우리가 시그모이드를 적용했는데 여기서는 렐루라는 걸 적용했습니다

렐루가 뭔지는 좀 있다 얘기 드리고요 일단은 시그모이드 비슷한 거다

이렇게 생각하시면 됩니다

그래서 그 출력이 나오면 그걸 리니어 2에다 넣고 그 다음에 여기서 그냥 시그모이드에 넣어 주시면 됩니다

그래서 여기 F.시그모이드 했는데 토치.NN.Functional 여기 가면은 우리가 이제 아까는 앞에서는 이렇게 시그모이드 이렇게 해서 만들어 놨던 거를 그냥 여기서 함수처럼 쓸 수가 있어요

일일이 만든 다음에 적용하려면 두 번 써야 되니까 귀찮잖아요

그래서 시그모이드 이런 애들은 굳이 여기서 만든 다음에 적용할 필요 없이 그냥 여기 F에 다 정의가 돼 있으니까 F.시그모이드 이렇게 하시면 그게 더 편합니다

그래서 이렇게 하시면 되고요 그래서 요거 또 코드를 한번 적용을 해 보면 적용을 해 보면 이렇게 되고 그럼 이걸로 이제 학습을 시켜야겠죠

그래서 학습을 시켜야 되니까 이걸로 이제 학습을 시켜야 되겠죠

학습을 어떻게 시키냐 하면 우리가 이제 모델은 요거는 이제 교재에 없는데 사실 아까 했던 거에 반복입니다

모델 이름이 MLP 클래시파이어 MLP 클래시파이어 이렇게 해 주시고 그 다음에 이제 우리가 손실 함수 정의해야 되는데 손실 함수는 똑같기 때문에 또 정의할 필요는 없지만 뭐 그래도 굳이 정의를 해 주면 NN.BCLoss로 해 주시면 될 거고 옵티밋으로 해 주시면 됩니다.

옵티마이저는 NN.BCLoss로 해 주시면 됩니다.

NN.BCLoss 아니었나 옵티마이저는 아 토치점 옵티미점 그냥 복사해서 쓰시면 됩니다

복사해서 쓰시면 되고 그 다음에 데이터 로더는 원래 있던 거 그대로 쓰면 되니까 또 해 줄 필요 없고요 그래서 트레인 로더 테스트하고 테스트 로더는 그냥 있는 거 그대로 쓰면 되고 그 다음에 이제 핏 함수 복사해 가지고 그대로 쓰시면 됩니다

그래서 요거만 추가해 주시고 크라이티리언이랑 옵티마이저 랑 핏이랑 요거는 원래 있던 코드 그대로 하시면 됩니다

그래서 이제 여기서부터는 우리가 이제 했던 코드가 있으니까 나머지는 재탕이에요.

이렇게 하면은 학습이 되죠


In [24]:
import torch.nn.functional as F

class MLPClassifier(nn.Module):
    def __init__(self, input_size=784):
        super().__init__()
        self.linear1 = nn.Linear(input_size, 16)  # 16개의 뉴런을 가진 은닉층
        self.linear2 = nn.Linear(16, 1)  # 한 개의 출력 뉴런
        
    def forward(self, x):
        x = x.view(-1, 784) # Flatten 레이어 없이, 텐서 모양을 (N, 28, 28) → (N, 784)
        x = F.relu(self.linear1(x)) # ReLU 활성화 함수
        x = F.sigmoid(self.linear2(x))  # 활성화 함수는 __init__에서 정의 X
        return x


In [ ]:
model = MLPClassifier() # 모형 만들기
criterion = nn.BCELoss()
optimizer = torch.optim.RAdam(model.parameters())
fit(model, train_loader, criterion, optimizer, epochs=3) # 전체 데이터를 3회 학습

In [26]:
import numpy as np
y_true = []  # 정답
y_prob = []  # 예측한 확률
with torch.no_grad():  # 그래디언트 계산 비활성화
    for x, y in test_loader:
        probs = model(x)
        y_true.extend(y.numpy())  # 정답에 추가
        y_prob.extend(probs.squeeze().numpy())  # 확률에 추가
y_prob = np.array(y_prob)  # 배열로 변환


In [27]:
threshold = 0.5  # 문턱값
y_pred = np.where(y_prob > threshold, 1, 0) # 문턱값보다 크면 1 아니면 0

# 혼동행렬 구하기
from sklearn.metrics import *
confusion_matrix(y_true, y_pred)


array([[925,  75],
       [ 31, 969]], dtype=int64)